Описание задачи
В некоторой архитектуре нейронной сети зависимость выхода $y \in \mathbb{R}$ от входа $(x_1, x_2) \in \mathbb{R}^2$ можно записать следующим образом:$$
\begin{align*}
&f_1(x_1, x_2) = x_1 + x_2,\\
&f_2(x_1, x_2) = x_1 \cdot x_2,\\
&g_1(x_1, x_2) = \tan( f_1(x_1, x_2) + f_2(x_1, x_2) + 100),\\
&g_2(x_1, x_2) = f_1(x_1, x_2) \cdot f_2(x_1, x_2),\\
&y(x_1, x_2) = p(g_1(x_1,x_2), g_2(x_1, x_2)),
\end{align*}
$$где $p(g_1, g_2)-$ некоторая неизвестная функция.

С помощью механизма автоматического дифференцирования в PyTorch необходимо вычислить значения частных производных в разных точках:$$
\frac{\partial y}{\partial x_1}, \frac{\partial y}{\partial x_2}, 
$$

Для этого в файле dev.json приведены значения $\frac{\partial p}{\partial g_1}, \frac{\partial p}{\partial g_2}$ (обозначены как dpdg1, dpdg2), вычисленные при известных значениях входа.

Нужно:

Загрузить данные из файла dev.json
Посчитать значения нужных производных для каждой точки.
Сохранить результат в формате CSV. Файл должен иметь следующую структуру:
id,dx1,dx2
....
id - это значение id из dev.json, dx1, dx2- значение производных, вычисленные в точке id из dev.json.
Ответ проверяется с помощью функции math.isclose с точностью до 1e-8.

In [2]:
import numpy as np
import pandas as pd
import torch

Решение $$\frac{\partial y}{\partial x_1}=\frac{\partial p}{\partial g_1}*\frac{\partial g_1}{\partial x_1}+\frac{\partial p}{\partial g_2}*\frac{\partial g_2}{\partial x_1}$$
$$ \frac{\partial y}{\partial x_2}=\frac{\partial p}{\partial g_1}*\frac{\partial g_1}{\partial x_2}+\frac{\partial p}{\partial g_2}*\frac{\partial g_2}{\partial x_2} $$

нам даны значения $$\frac{\partial p}{\partial g_1} и  \frac{\partial p}{\partial g_2} $$
найдем с помощью autograd значения $$ \frac{\partial g_1}{\partial x_1},\frac{\partial g_1}{\partial x_2},\frac{\partial g_2}{\partial x_1},\frac{\partial g_2}{\partial x_2} $$
и задача будет решена.

Загружаем файл с данными:

In [7]:
dev_data=pd.read_json('/content/drive/MyDrive/Colab Notebooks/Test 3dML/Task2/dev.json')

Определяем функции:

In [43]:
def f1 ( x1 , x2 ):
  return x1 + x2
def f2 ( x1 , x2 ):
  return x1 * x2
def g1 ( x1 , x2 ):
  return torch.tan( x1 + x2 + 100)

Задаем значения $ x_1, x_2 $ и определяем через них функцию $ g_1(f_1(x_1,x_2),f_2(x_1,x_2)) $

In [73]:
x1=torch.tensor(dev_data['x1'],requires_grad=True)
x2=torch.tensor(dev_data['x2'],requires_grad=True)
ff1=f1(x1,x2)
ff2=f2(x1,x2)
fg1=g1(ff1,ff2)

Вычисляем градиент

In [74]:
fg1.backward(torch.ones(10))

Записываем в таблицу dev_data $\frac{\partial g_1}{\partial x_1},\frac{\partial g_1}{\partial x_2}$ в колонки 'dg1dx1' и 'dg1dx2' соответственно.

In [80]:
dev_data['dg1dx1']=x1.grad.numpy()
dev_data['dg1dx2']=x2.grad.numpy()

Аналогичным образом вычисляем и записываем в таблицу $\frac{\partial g_2}{\partial x_1},\frac{\partial g_2}{\partial x_2}$

In [82]:
x1=torch.tensor(dev_data['x1'],requires_grad=True)
x2=torch.tensor(dev_data['x2'],requires_grad=True)
ff1=f1(x1,x2)
ff2=f2(x1,x2)
fg2=f2(ff1,ff2)

In [84]:
fg2.backward(torch.ones(10))

In [85]:
dev_data['dg2dx1']=x1.grad.numpy()
dev_data['dg2dx2']=x2.grad.numpy()

В итоге в нашей таблице есть все необходимые данные для вычисления $\frac{\partial y}{\partial x_1},\frac{\partial y}{\partial x_2}$

In [87]:
dev_data.head(3)

,id,x1,x2,dpdg1,dpdg2,dg1dx1,dg1dx2,dg2dx1,dg2dx2
0,1,0.578468,0.252695,0.957181,0.242992,3.079246,3.880027,0.356207,0.626977
1,2,0.180040,0.857373,1.554807,0.320274,5.959775,3.786410,1.043811,0.341138
2,3,0.858978,0.597631,15.086440,1.495505,185.006195,215.270279,1.383866,1.764546


Вычисляем необходимые производные:

In [88]:
dev_data['dx1']=dev_data['dpdg1']*dev_data['dg1dx1']+dev_data['dpdg2']*dev_data['dg2dx1']
dev_data['dx2']=dev_data['dpdg1']*dev_data['dg1dx2']+dev_data['dpdg2']*dev_data['dg2dx2']

Записываем результирующий файл:

In [93]:
dev_data[['id','dx1','dx2']].to_csv('/content/drive/MyDrive/Colab Notebooks/Test 3dML/Task2/response.csv',index=False)